In [ ]:
import pandas as pd
df = pd.read_csv(r"C:\Users\somya\cleaned_credit_reports.csv")
print(df.head())
df.fillna(df.select_dtypes(include=["number"]).median(), inplace=True)



  Customer ID  Credit Score  Active Loans  Credit Utilization (%)  \
0    CUST3141           567             3                   40.86   
1    CUST4357           565             1                   71.55   
2    CUST3838           747             0                   75.38   
3    CUST2343           717             0                   43.45   
4    CUST8876           597             2                   43.44   

   Missed Payments (12 Months)  Total Debt  Debt-to-Income Ratio  
0                            4   161854.14                  0.32  
1                            3   166507.41                  0.38  
2                            2   184491.38                  0.46  
3                            1   117802.79                  0.61  
4                            2    34003.18                  0.79  


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
print(df.columns)
if "Risk" not in df.columns:
    df["Risk"] = df["Credit Score"].apply(lambda x: 1 if x < 600 else 0)
features = ["Credit Score", "Debt-to-Income Ratio", "Missed Payments (12 Months)", "Credit Utilization (%)", "Total Debt"]
X = df[features].copy()
y = df["Risk"]
X = X.fillna(X.mean())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=features)
print(X_scaled.head())
print(y.head())


Index(['Customer ID', 'Credit Score', 'Active Loans', 'Credit Utilization (%)',
       'Missed Payments (12 Months)', 'Total Debt', 'Debt-to-Income Ratio'],
      dtype='object')
   Credit Score  Debt-to-Income Ratio  Missed Payments (12 Months)  \
0     -0.921726             -1.131843                     1.568929   
1     -0.947472             -0.778142                     0.588348   
2      1.395462             -0.306541                    -0.392232   
3      1.009264              0.577711                    -1.372813   
4     -0.535528              1.638814                    -0.392232   

   Credit Utilization (%)  Total Debt  
0               -0.925651    0.534538  
1                1.092553    0.620538  
2                1.344417    0.952915  
3               -0.755330   -0.279611  
4               -0.755988   -1.828380  
0    1
1    1
2    0
3    0
4    1
Name: Risk, dtype: int64


In [ ]:
!pip install imbalanced-learn


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print("Training set class distribution:\n", y_train.value_counts())
print("Test set class distribution:\n", y_test.value_counts())
if y_train.value_counts().min() < 10:
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=1))


Training set class distribution:
 Risk
1    2
0    2
Name: count, dtype: int64
Test set class distribution:
 Risk
1    1
Name: count, dtype: int64
Accuracy: 0.0
              precision    recall  f1-score   support

           0       0.00      1.00      0.00       0.0
           1       1.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.50      0.50      0.00       1.0
weighted avg       1.00      0.00      0.00       1.0

